# KNN Classification and Logistic Regression on UWaveGesture Data
In this project, our aim is to perform knn classification with the most suitable k value and logistic regression with penalty and without penalty to predict the classes of test data instances.

                                                                                    Alperen Köksal

Required libraries are imported

In [131]:
#Required Libraries
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from scipy.spatial import distance
from scipy.spatial import distance_matrix
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS
from sklearn.metrics import pairwise_distances
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
import time

UWaveGesture data is concatenated in a 3T length of columns.

In [6]:
gest_train_x = pd.read_csv("C:/Users/a_kok/Desktop/Dersler/IE 582/HW2/uWaveGestureLibrary_X_TRAIN",header=None, sep=" ",
                             prefix="", skipinitialspace=True)
gest_train_y = pd.read_csv("C:/Users/a_kok/Desktop/Dersler/IE 582/HW2/uWaveGestureLibrary_Y_TRAIN",header=None, sep=" ",
                             prefix="", skipinitialspace=True)
gest_train_z = pd.read_csv("C:/Users/a_kok/Desktop/Dersler/IE 582/HW2/uWaveGestureLibrary_Z_TRAIN",header=None, sep=" ",
                             prefix="", skipinitialspace=True)
gest_train_y.drop(columns = 0,inplace = True)
gest_train_z.drop(columns = 0,inplace = True)
gest_train_concat = pd.concat([gest_train_x, gest_train_y,gest_train_z], axis=1)
gest_train_concat.rename(columns = {0:"class"},inplace = True)
trainclass = gest_train_concat["class"]
gest_train_concat = gest_train_concat.drop(columns = "class")

In [7]:
gest_test_x = pd.read_csv("C:/Users/a_kok/Desktop/Dersler/IE 582/HW2/uWaveGestureLibrary_X_TEST",header=None, sep=" ",
                             prefix="", skipinitialspace=True)
gest_test_y = pd.read_csv("C:/Users/a_kok/Desktop/Dersler/IE 582/HW2/uWaveGestureLibrary_Y_TEST",header=None, sep=" ",
                             prefix="", skipinitialspace=True)
gest_test_z = pd.read_csv("C:/Users/a_kok/Desktop/Dersler/IE 582/HW2/uWaveGestureLibrary_Z_TEST",header=None, sep=" ",
                             prefix="", skipinitialspace=True)
gest_test_y.drop(columns = 0,inplace = True)
gest_test_z.drop(columns = 0,inplace = True)
gest_test_concat = pd.concat([gest_test_x, gest_test_y,gest_test_z], axis=1)
gest_test_concat.rename(columns = {0:"class"},inplace = True)
testclass = gest_test_concat["class"]
testclass = pd.DataFrame(testclass)
gest_test_concat = gest_test_concat.drop(columns = "class")

In [8]:
gest_train_concat

,1,2,3,4,5,6,7,8,9,10,...,306,307,308,309,310,311,312,313,314,315
0,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,-0.304243,...,0.523217,0.513994,0.503481,0.492967,0.474522,0.456077,0.437632,0.419187,0.400743,0.382298
1,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,1.627311,...,-0.427010,-0.427010,-0.427010,-0.427172,-0.428773,-0.440720,-0.452667,-0.464613,-0.476560,-0.488507
2,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,0.661277,...,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717,-0.862717
3,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,0.005185,...,-0.187384,-0.123549,-0.055870,0.011808,0.079487,0.157056,0.253740,0.445503,0.648538,0.851573
4,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,1.286198,...,1.867473,1.834319,1.756000,1.638582,1.521164,1.453266,1.515219,1.632637,1.750054,1.867473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,0.117811,0.117811,0.117811,0.117811,0.117811,0.117811,0.117811,0.117811,0.117811,0.117811,...,-0.212484,-0.137582,-0.066376,-0.010642,0.026809,0.064260,0.101712,0.139163,0.176614,0.214065
892,-1.232590,-1.232590,-1.232590,-1.232590,-1.232590,-1.232590,-1.232590,-1.232590,-1.232590,-1.232590,...,0.222775,0.234980,0.247184,0.259389,0.271593,0.283798,0.296003,0.308207,0.320412,0.332617
893,0.282877,0.282877,0.282877,0.282877,0.282877,0.282877,0.282877,0.282877,0.282877,0.282877,...,-0.044573,-0.043387,-0.038164,0.000695,0.027133,0.039175,0.088921,0.158608,0.223306,0.267828
894,1.248704,1.248704,1.248704,1.248704,1.248704,1.248704,1.248704,1.248704,1.248704,1.248704,...,-0.160172,-0.162960,-0.190327,-0.296460,-0.407102,-0.504592,-0.559242,-0.549343,-0.505912,-0.433044


In [9]:
gest_test_concat

,1,2,3,4,5,6,7,8,9,10,...,306,307,308,309,310,311,312,313,314,315
0,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,1.557018,...,-0.348423,-0.348423,-0.344675,-0.340728,-0.333147,-0.325423,-0.317700,-0.309976,-0.302252,-0.294528
1,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,-0.075096,...,0.961350,0.849142,0.725546,0.589277,0.539075,0.515810,0.515810,0.515810,0.515810,0.515810
2,1.509012,1.526296,1.543580,1.560865,1.573811,1.576913,1.578592,1.578592,1.578592,1.578592,...,-0.565962,-0.475680,-0.399089,-0.344074,-0.289058,-0.234042,-0.179026,-0.124011,-0.068995,-0.013979
3,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,1.303974,...,1.044950,1.044950,1.044950,1.044950,1.079339,1.115085,1.171001,1.227442,1.283883,1.340324
4,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,1.298089,...,-0.693460,-0.689765,-0.686070,-0.682375,-0.678680,-0.674985,-0.671290,-0.667595,-0.663900,-0.660205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577,0.157285,0.157285,0.157285,0.157285,0.157285,0.157285,0.157285,0.157285,0.157285,0.157285,...,-0.212601,-0.202155,-0.189816,-0.149002,-0.154077,-0.180674,-0.228201,-0.256934,-0.249682,-0.202155
3578,0.145031,0.145031,0.145031,0.145031,0.145031,0.145031,0.175495,0.214111,0.241395,0.241395,...,0.249209,0.196203,0.142721,0.099317,0.075075,0.075075,0.075075,0.075075,0.075075,0.075075
3579,-1.028283,-1.028283,-1.028283,-1.028283,-1.028283,-1.028283,-1.028283,-1.028283,-1.028283,-1.028283,...,0.272432,0.243039,0.213647,0.186249,0.166012,0.159199,0.159199,0.159199,0.159199,0.159199
3580,-0.755221,-0.755221,-0.755221,-0.755221,-0.755221,-0.755221,-0.755221,-0.755221,-0.755221,-0.755221,...,-0.439584,-0.414049,-0.388513,-0.359231,-0.309808,-0.258860,-0.207789,-0.164342,-0.131842,-0.106307


### Task 1 – On the use of distance information for UwaveGesture Recognition Task
#### a)
Two distance measures are used: manhattan and euclidean to perform the Knn classification. best k values are determined by using cross validation with 10 fold on train data. Knn classification model is fit with this k value to predict the classes of test instances


In [11]:
np.random.seed(500)
knn_manhattan = KNeighborsClassifier(p=1)
knn_euclidean = KNeighborsClassifier(p=2)

In [12]:
param_grid={'n_neighbors': list(range(1,26))}

In [13]:
knn_gscv_manhattan = GridSearchCV(knn_manhattan, param_grid, cv=10)
knn_gscv_euclidean = GridSearchCV(knn_euclidean, param_grid, cv=10)

In [14]:
knn_gscv_manhattan.fit(gest_train_concat, trainclass)
knn_gscv_euclidean.fit(gest_train_concat, trainclass)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25]})

Best k values are 1 and 3 for the manhattan and euclidean distance knn classification

In [15]:
print(knn_gscv_manhattan.best_params_)
print(knn_gscv_manhattan.best_score_)

{'n_neighbors': 1}
0.9587390761548065


In [16]:
print(knn_gscv_euclidean.best_params_)
print(knn_gscv_euclidean.best_score_)

{'n_neighbors': 3}
0.9441947565543071


In [133]:
start = time.time()
testclass["predicted_manhattan"] = knn_gscv_manhattan.predict(gest_test_concat)
confusion_matrix_manhattan = metrics.confusion_matrix(testclass["class"],testclass["predicted_manhattan"])
end = time.time()
end-start

3.946470022201538

In [134]:
start = time.time()
testclass["predicted_euclidean"] = knn_gscv_euclidean.predict(gest_test_concat)
confusion_matrix_euclidean = metrics.confusion_matrix(testclass["class"],testclass["predicted_euclidean"])
end = time.time()
end-start

0.28922557830810547

#### b)
Runtime is much higher for manhattan distance than euclidean distance. Accuracy of manhattan distance is higher than euclidean distance since dimensionality is very high.

In [19]:
confusion_matrix_manhattan = pd.DataFrame(confusion_matrix_manhattan,index = np.arange(1,9),columns = np.arange(1,9))
confusion_matrix_manhattan
accuracy_manhattan = metrics.accuracy_score(testclass["class"],testclass["predicted_manhattan"])
print(accuracy_manhattan)
confusion_matrix_manhattan

0.954215522054718


,1,2,3,4,5,6,7,8
1,430,0,0,2,0,5,0,0
2,1,451,0,0,0,0,0,0
3,2,0,418,0,16,14,4,0
4,3,0,0,392,40,11,0,4
5,3,0,9,5,415,1,0,0
6,3,0,5,13,16,411,0,1
7,0,0,2,0,0,0,445,0
8,0,0,0,3,1,0,0,456


In [20]:
confusion_matrix_euclidean = pd.DataFrame(confusion_matrix_euclidean,index = np.arange(1,9),columns = np.arange(1,9))
accuracy_euclidean = metrics.accuracy_score(testclass["class"],testclass["predicted_euclidean"])
print(accuracy_euclidean)
confusion_matrix_euclidean

0.9438860971524288


,1,2,3,4,5,6,7,8
1,431,0,0,2,0,4,0,0
2,1,449,0,0,0,0,2,0
3,2,0,413,0,15,20,4,0
4,7,0,0,370,60,6,0,7
5,3,0,6,1,422,1,0,0
6,6,0,7,15,28,392,1,0
7,0,0,1,0,0,0,446,0
8,0,0,0,1,1,0,0,458


#### c)
I think it can be reasonable since two different axes are dominant in each instances such as: x,y y,z x,z. In other words, none of axes is dominant in determining the class of all instances.

### Task 2 – Linear models on alternative representations of the data
#### a)
Logistic regression without any penalty will be used to predict if test instance x belongs to class 3 or not.

In [40]:
trainclass_regression = (trainclass == 3)*1
class3_ratio = trainclass_regression.sum()/len(trainclass)

In [75]:
lr_first = LogisticRegression(max_iter = 10000,penalty = "none")
lr_first.fit(gest_train_concat,trainclass_regression)

LogisticRegression(max_iter=10000, penalty='none', random_state=500)

In [76]:
lr_first_prediction = lr_first.predict_proba(gest_test_concat)
lr_first_prediction = pd.DataFrame(lr_first_prediction)
lr_first_prediction["prediction"] = lr_first_prediction[1].apply(lambda x:1 if x > class3_ratio 
                                                              else(0))
testclass["prediction_lr"] = lr_first_prediction["prediction"]

In [80]:
confusion_matrix_lr = metrics.confusion_matrix((testclass["class"] == 3)*1,testclass["prediction_lr"])
confusion_matrix_lr = pd.DataFrame(confusion_matrix_lr)
accuracy_lr = metrics.accuracy_score((testclass["class"] == 3)*1,testclass["prediction_lr"])
print(confusion_matrix_lr)
accuracy_lr

      0    1
0  2953  175
1   165  289


0.9050809603573423

#### b)
Logistic regression with lasso penaly will be used to predict if test instance x belongs to class 3 or not. Best lambda value will be determined by using 10 fold cross validation

In [101]:
lr_second = LogisticRegressionCV(random_state = 500,cv= 10, max_iter = 10000,penalty = "l1",solver = "liblinear",Cs=np.arange(0.5,10,0.5),
                                scoring = "neg_log_loss")
lr_second.fit(gest_train_concat,trainclass_regression)

LogisticRegressionCV(Cs=array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
       7. , 7.5, 8. , 8.5, 9. , 9.5]),
                     cv=10, max_iter=10000, penalty='l1', random_state=500,
                     scoring='neg_log_loss', solver='liblinear')

In [107]:
lr_second.C_

array([0.5])

In [108]:
lr_second = LogisticRegressionCV(random_state = 500,cv= 10, max_iter = 10000,penalty = "l1",solver = "liblinear",Cs=np.arange(0.1,2,0.05),
                                scoring = "neg_log_loss")
lr_second.fit(gest_train_concat,trainclass_regression)

LogisticRegressionCV(Cs=array([0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55, 0.6 ,
       0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.05, 1.1 , 1.15,
       1.2 , 1.25, 1.3 , 1.35, 1.4 , 1.45, 1.5 , 1.55, 1.6 , 1.65, 1.7 ,
       1.75, 1.8 , 1.85, 1.9 , 1.95]),
                     cv=10, max_iter=10000, penalty='l1', random_state=500,
                     scoring='neg_log_loss', solver='liblinear')

In [109]:
lr_second.C_

array([0.45])

In [110]:
lr_second_prediction = lr_second.predict_proba(gest_test_concat)
lr_second_prediction = pd.DataFrame(lr_second_prediction)
lr_second_prediction["prediction"] = lr_second_prediction[1].apply(lambda x:1 if x > class3_ratio 
                                                              else(0))
testclass["prediction_lr_lasso"] = lr_second_prediction["prediction"]

In [111]:
confusion_matrix_lr_lasso = metrics.confusion_matrix((testclass["class"] == 3)*1,testclass["prediction_lr_lasso"])
confusion_matrix_lr_lasso = pd.DataFrame(confusion_matrix_lr_lasso)
accuracy_lr_lasso = metrics.accuracy_score((testclass["class"] == 3)*1,testclass["prediction_lr_lasso"])
print(confusion_matrix_lr_lasso)
accuracy_lr_lasso

      0    1
0  2869  259
1    30  424


0.9193188163037409

#### c)
Logistic regression with lasso penaly will be used on distance matrix of concatenated train data to predict if test instance x belongs to class 3 or not. Best lambda value will be determined by using 10 fold cross validation

In [124]:
gest_train_concat_dist_matrix = distance_matrix(gest_train_concat,gest_train_concat)
gest_test_concat_dist_matrix = distance_matrix(gest_test_concat,gest_train_concat)

In [125]:
gest_test_concat_dist_matrix = pd.DataFrame(gest_test_concat_dist_matrix)
gest_train_concat_dist_matrix = pd.DataFrame(gest_train_concat_dist_matrix)

In [121]:
lr_last= LogisticRegressionCV(random_state = 500,cv= 10, max_iter = 10000,penalty = "l1",solver = "liblinear",Cs=np.arange(0.5,10,0.5),
                                scoring = "neg_log_loss")
lr_last.fit(gest_train_concat_dist_matrix,trainclass_regression)

LogisticRegressionCV(Cs=array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
       7. , 7.5, 8. , 8.5, 9. , 9.5]),
                     cv=10, max_iter=10000, penalty='l1', random_state=500,
                     scoring='neg_log_loss', solver='liblinear')

In [122]:
lr_last.C_

array([3.5])

In [126]:
lr_last_prediction = lr_last.predict_proba(gest_test_concat_dist_matrix)
lr_last_prediction = pd.DataFrame(lr_last_prediction)
lr_last_prediction["prediction"] = lr_last_prediction[1].apply(lambda x:1 if x > class3_ratio 
                                                              else(0))
testclass["prediction_lr_lasso_matrix"] = lr_last_prediction["prediction"]

In [127]:
confusion_matrix_lr_lasso_distance = metrics.confusion_matrix((testclass["class"] == 3)*1,testclass["prediction_lr_lasso_matrix"])
confusion_matrix_lr_lasso_distance = pd.DataFrame(confusion_matrix_lr_lasso_distance)
accuracy_lr_lasso_distance = metrics.accuracy_score((testclass["class"] == 3)*1,testclass["prediction_lr_lasso_matrix"])
print(confusion_matrix_lr_lasso_distance)
accuracy_lr_lasso_distance

      0    1
0  3099   29
1    23  431


0.9854829704075935

Knn classification methods have better performance than logistic regression methods overall. Accuracy values are comparably high which means those models are suitable to predict the class of test instances. On the other hand distance matrix featured logistic regression method with lasso penalty has the best accuracy.

To conclude, Knn classification and logistic regression models are used on UWaveGesture data to predict the class information. Knn classification methods had better performance than logistic regression methods but logistic regression on distance matrix of UWaveGesture matrix had the best performance.